<a href="https://colab.research.google.com/github/cisprague/bts-2021/blob/main/notebooks/bts_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# System identification of AUVs with physics-informed learning

Christopher Iliffe Sprague (sprague@kth.se)

Sriharsha Bhat (svbhat@kth.se)

# Outline

- Background
- What's the benefit?
- Data generation
- Model identification
- Simulation
- Discussion and future work

# Resources

In [2]:
# JAX for autograd and and numerics
! pip install --upgrade pip
! pip install --upgrade "jax[cuda101]" -f https://storage.googleapis.com/jax-releases/jax_releases.html
import jax.numpy as np
from jax import jit, vmap, jacobian, hessian, grad
from jax.experimental.ode import odeint
from jax.random import uniform, choice
from jax.nn import softplus
from jax.lax import stop_gradient

# Haiku for neural networks
! pip install git+https://github.com/deepmind/dm-haiku
import haiku as hk

# Optax for optimisation
! pip install git+https://github.com/deepmind/optax
import optax

# TQDM for training loop feedback
! pip install tqdm
from tqdm.notebook import tqdm

# SKLearn for preprocessing data
from sklearn import preprocessing

# treating some arguments as static
from functools import partial

# matplotlib for plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://storage.googleapis.com/jax-releases/jax_releases.html
     |████████████████████████████████| 165.4 MB 1.1 kB/s 
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.69+cuda111
    Uninstalling jaxlib-0.1.69+cuda111:
      Successfully uninstalled jaxlib-0.1.69+cuda111
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-7myft_i0
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-7myft_i0
  Resolved https://github.com/deepmind/dm-haiku to commit 4ae60fd4fd2da3b2f8f9ad3ec6dfd893745b483b
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/deepmind/optax to /tmp/pip-req-build-8n9vr5vw
  Running command git 

In [3]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

cpu


# Analytical model
We will gather data from an analytical model of an AUV, so that we can compare the learned/identified model with a ground truth. However, in practice, one can also use other data as input, such as AUV pose data from missions.

A six degree-of freedom AUV model is used leading to 12 states ($x,y,z,\phi,\theta,\psi,u,v,w,p,q,r $), based on Fossen's notation. 

The kinematics are of the form
    $\dot \eta = J_{euler}(\eta)\nu$ , 
    where $\eta$ represents the vector of positions and orientations in Euler angles, while $\nu$ represents the vector of velocities and angular velocities and $J_{euler}\in\mathbb{R}^{6 \times 6}$ is the kinematic transformation matrix. 

The dynamics are of the form
    $M_{\text{eff}} \dot\nu + C_{\text{eff}}(\nu) + g(\eta) = \tau_C$, 
    where M represents the combined mass and inertia matrix (considering rigid body terms, but added mass terms can also be included), C is the combined coriolis-centripetal matrix, D is the damping matrix (considering hydrodynamics), g represents the buoyancy forces (considering hydrostatics) and $\tau_C$ refers to the vector of external control forces (such as from actuators).

The AUV under consideration (SAM) has 6 actuators including counter-rotating propellers (to control speed and roll), thrust vectoring(for yaw and pitch), longitudinal center of gravity(for static pitching and gliding) and a variable buoyancy system (to regulate buoyancy) and is underactuated. The effect of these actuators are included in the code below.


In [2]:
state_dim = 12
control_dim = 6

In [3]:
@jit
def skew(l):
  return np.array([
    [0, -l[2], l[1]],
    [l[2], 0, -l[0]],
    [-l[1], l[0], 0]
  ])

In [4]:
@jit
def f_analytical(state, control):

  # state and control
  x, y, z, phi, theta, psi, u, v, w, p, q, r = state
  rpm1, rpm2, de, dr, lcg, vbs = control

  # position and velocity
  eta = np.array([x, y, z, phi, theta, psi])
  nu = np.array([u, v, w, p, q, r])

  # scaled controls
  rpm1 *= 1000.0
  rpm2 *= 1000.0
  de *= 0.05
  dr *= 0.05
  # vbs *= 1.0
  # lcg *= 1.0

  # mass and inertia matrix
  m = 14.0
  I_o = np.diag(np.array([0.0294, 1.6202, 1.6202]))

  # centre of gravity, buoyancy, and pressure positions, resp.
  r_g = np.array([0.1 + lcg*0.01, 0.0, 0.0])
  r_b = np.array([0.1, 0.0, 0.0])
  r_cp = np.array([0.1, 0.0, 0.0])

  # <insert title>
  W = m*9.81
  B = W + vbs*1.5

  # hydrodynamic coefficients
  Xuu = 5. #3. #1.0
  Yvv = 20. #10. #100.0
  Zww = 50. #100.0
  Kpp = 0.1 #10.0
  Mqq = 20.#40 #100.0
  Nrr = 20. #150.0

  # control actuators
  K_T = np.array([0.0175, 0.0175])
  Q_T = np.array([0.001, -0.001])#*0.0

  # mass and inertia matrix
  M = np.block([
    [m*np.eye(3,3), -m*skew(r_g)],
    [m*skew(r_g), I_o]
  ])
  assert M.shape == (6,6), M

  # coriolis and centripetal matrix
  nu1 = np.array([u, v, w])
  nu2 = np.array([p, q, r])
  top_right = -m*skew(nu1) - m*skew(nu2)*skew(r_g)
  bottom_left = -m*skew(nu1) + m*skew(r_g)*skew(nu2)
  bottom_right = -skew(I_o.dot(nu2))
  C_RB = np.block([
    [np.zeros((3,3)), top_right],
    [bottom_left, bottom_right]
  ])
  assert C_RB.shape == (6, 6), C_RB

  # damping matrix
  forces = np.diag(np.array([Xuu*np.abs(u), Yvv*np.abs(v), Zww*np.abs(w)]))
  moments = np.diag(np.array([Kpp*np.abs(p), Mqq*np.abs(q), Nrr*np.abs(r)]))
  coupling = np.matmul(skew(r_cp), forces)
  D = np.block([[forces, np.zeros((3, 3))], [-coupling, moments]])
  assert D.shape == (6, 6), D

  # rotational transform between body and NED in Euler        
  T_euler = np.array([
    [1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],
    [0, np.cos(phi), -np.sin(phi)],
    [0, np.sin(phi)/np.cos(theta), np.cos(phi)/np.cos(theta)],
  ])
  R_euler = np.array([
    [
      np.cos(psi)*np.cos(theta),
      -np.sin(psi)*np.cos(phi)+np.cos(psi)*np.sin(theta)*np.sin(phi),
      np.sin(psi)*np.sin(phi)+np.cos(psi)*np.cos(phi)*np.sin(theta)
    ],
    [
      np.sin(psi)*np.cos(theta),
      np.cos(psi)*np.cos(phi)+np.sin(phi)*np.sin(theta)*np.sin(psi),
      -np.cos(psi)*np.sin(phi)+np.sin(theta)*np.sin(psi)*np.cos(phi),
    ],
    [
      -np.sin(theta),
      np.cos(theta)*np.sin(phi),
      np.cos(theta)*np.cos(phi),
    ],
  ])
  assert R_euler.shape == (3,3), R_euler
  J_eta = np.block([
    [R_euler, np.zeros((3,3))],
    [np.zeros((3,3)), T_euler]
  ])
  assert J_eta.shape == (6,6), J_eta

  # buoyancy in quaternions
  f_g = np.array([0, 0, W])
  f_b = np.array([0, 0, -B])
  row1 = np.linalg.inv(R_euler).dot(f_g + f_b)
  row2 = skew(r_g).dot(np.linalg.inv(R_euler)).dot(f_g) + \
    skew(r_b).dot(np.linalg.inv(R_euler)).dot(f_b)
  geta = np.block([row1, row2])
  assert geta.shape == (6,), geta

  # <insert title>
  F_T = K_T.dot(np.array([rpm1, rpm2]))
  M_T = Q_T.dot(np.array([rpm1, rpm2]))
  tauc = np.array([
    F_T*np.cos(de)*np.cos(dr),
    -F_T*np.sin(dr),
    F_T*np.sin(de)*np.cos(dr),
    M_T*np.cos(de)*np.cos(dr),
    -M_T*np.sin(dr),
    M_T*np.sin(de)*np.cos(dr)
  ])
  assert tauc.shape == (6,), tauc

  # velocity and acceleration 
  etadot = np.block([J_eta.dot(nu)])
  assert etadot.shape == (6,)
  nudot = np.linalg.inv(M).dot(tauc - (C_RB + D).dot(nu - geta))
  assert nudot.shape == (6,)

  # state-space
  sdot = np.block([etadot, nudot])
  return sdot

# Generating data

We want to ensure percitant excitation to make sure we cover a variety of states.

To do this, we will simulate random state-feedback controllers. A straighforward way to do this is to simulate NN controllers with random parameters.

## Integration

In [ ]:
# a single integration step for a single state
def rk4_step(f, x, u, h):
  k1 = f(x, u)
  k2 = f(x + h*k1/2, u)
  k3 = f(x + h*k2/2, u)
  k4 = f(x + h*k3, u)
  return x + h*(k1 + 2*k2 + 2*k3 + k4)/6

## Random training data

In [5]:
key = hk.PRNGSequence(42)
n_train = 100
h = 0.01

In [ ]:
states0 = uniform(
    next(key),
    shape=(n_train, state_dim),
    minval=np.array([-10, -10, -10, -np.pi, -np.pi, -np.pi, -5, -5, -5, -5, -5, -5]),
    maxval=np.array([10, 10, 10, np.pi, np.pi, np.pi, 5, 5, 5, 5, 5, 5])
)

In [ ]:
controls0 = uniform(
    next(key),
    shape=(n_train, control_dim),
    minval=np.full(control_dim, -1, dtype=np.float32),
    maxval=np.full(control_dim, 1, dtype=np.float32)
)

In [ ]:
dstates0 = vmap(f_analytical, in_axes=(0, 0))(states0, controls0)

In [ ]:
states1 = rk4_steps_analytical(states0, controls0, h)

In [ ]:
scaler = preprocessing.StandardScaler().fit(states0)
mean = scaler.mean_
scale = scaler.scale_
mean = 0
scale = 1

## Visualisation

In [ ]:
# TODO: add quiver
trajs = np.hstack((states0, states1)).reshape((-1, 2, 12))
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(1000):
  ax.plot(trajs[i,:,0], trajs[i,:,1], trajs[i,:,2], 'k-')
plt.show()

# Learning models

We will test a naïve multi-layer-perceptron model and a physics-informed model (https://arxiv.org/pdf/2106.00026.pdf).

## Naïve model

In [ ]:
def F_naive(state, control):
  state = (state - mean)/scale
  net = hk.nets.MLP([500, 500, 500, 12], activation=softplus)
  return net(np.hstack((state, control)))

## Physics-informed model

To learn the dynamics model from data we will use physics-informed learning: a way to constrain neural network models to obey the first-principles of physics.

We will model a Lagrangian to learn to conservative part of the dynamics.


$$
\ddot q = (\nabla_{\dot q}\nabla_{\dot q}^{\top}\mathcal{L})^{-1}[\nabla_q \mathcal{L} - (\nabla_{q}\nabla_{\dot q}^{\top}\mathcal{L})\dot q
$$



In [ ]:
def lagrangian(q, dq):
  net = hk.nets.MLP([200, 200, 200, 1], activation=softplus)
  return net(np.hstack((q, dq)))[0]

In [ ]:
def F_conservative(state):
  q, dq = np.split(state, 2)
  ddq = jacobian(jacobian(lagrangian, 1), 0)(q, dq)
  ddq = ddq @ dq
  ddq = grad(lagrangian, 0)(q, dq) - ddq
  ddq = np.linalg.pinv(hessian(lagrangian, 1)(q, dq)) @ ddq
  return np.hstack((dq, ddq))

In [ ]:
def F_non_conservative(state, control):
  net = hk.nets.MLP([200, 200, 200, 12], activation=softplus)
  return net(np.hstack((state, control)))

In [ ]:
def F_decomposition(state, control):
  return f_conservative(state) + f_non_conservative(state, control)

# Loss function design

Let us assume that we will have a database of trajectories, either obtained through simulation of an analytical model or through robotic field data.



In [ ]:
@partial(jit, static_argnums=(0,2))
def loss(f_nn, params, n, key):

    # generate random states
    states0 = uniform(
        key,
        shape=(n, state_dim),
        minval=np.array([-10, -10, -10, -np.pi, -np.pi, -np.pi, -5, -5, -5, -5, -5, -5]),
        maxval=np.array([10, 10, 10, np.pi, np.pi, np.pi, 5, 5, 5, 5, 5, 5])
    )

    # generate random controls
    controls0 = uniform(
        key,
        shape=(n, control_dim),
        minval=np.full(control_dim, -1, dtype=np.float32),
        maxval=np.full(control_dim, 1, dtype=np.float32)
    )

    # compute analytical time-derivative of state
    # NOTE: assumes velocity and acceleration are known from field data
    dstates0 = vmap(f_analytical, in_axes=(0,0))(states0, controls0)

    # compute NN time-derivative predictions
    nn_dstates0 = vmap(f_nn.apply, in_axes=(None,0,0))(params, states0, controls0)

    # mean-squared error loss
    loss = np.average(np.square(dstates0 - nn_dstates0))
    return loss, loss


In [ ]:
loss_grad = jit(grad(loss, argnums=1, has_aux=True), static_argnums=(0,2))

## Naïve model

In [ ]:
f_naive = hk.without_apply_rng(hk.transform(F_naive))
params_naive = f_naive.init(next(key), states0[0,:], controls0[0,:])

In [ ]:
@jit
def rk4_step_naive(params, x, u, h):
  # f = lambda x, u: f_naive.apply(params, (x - mean)/scale, u)
  return rk4_step(lambda x, u: f_naive.apply(params, x, u), x, u, h)
rk4_steps_naive = jit(vmap(rk4_step_naive, in_axes=(None, 0, 0, None)))

In [ ]:

def loss_naive(params):

  # naïve dynamics
  # nn_states1 = rk4_steps_naive(params, states0, controls0, h)
  nn_dstates0 = vmap(f_naive.apply, (None, 0, 0))(params, states0, controls0)

  # MSE
  # loss = np.average(np.square(nn_states1 - states1))
  loss = np.average(np.square(dstates0 - nn_dstates0))
  return loss, loss

In [ ]:
# gradient of loss function
grad_loss_naive = jit(grad(loss_naive, argnums=0, has_aux=True))

# Training

In [ ]:
# setup optimiser and initialise its state
opt_init, opt_update = optax.chain(optax.scale_by_adam(), optax.scale(-1e-3))
opt_state = opt_init(params_naive)

In [ ]:
# loss data
losses_naive = list()

In [ ]:
# training loop
pb = tqdm(range(1000))
for _ in pb:

  # gradient and loss
  gopt, loss = loss_grad(f_naive, params_naive, 500, next(key))
  # gopt, loss = grad_loss_naive(params_naive)

  # progress bar update
  pb.set_description('{:.6f}'.format(loss))

  # optimiser update
  updates, opt_state = opt_update(gopt, opt_state, params_naive)

  # update the parameters
  params_naive = optax.apply_updates(params_naive, updates)

# Results

## States and times

In [ ]:
times = np.arange(0.0, 10, h)
n_trajs = 2
i = choice(
  next(key), 
  np.arange(0, n_train), 
  shape=(n_trajs,)
)
states = states0[i, :]

## Random control policy

In [ ]:
# random NN controller
def Policy(state):
  net = hk.nets.MLP([200, 200, control_dim])
  return np.tanh(net(state))

In [ ]:
policy = hk.without_apply_rng(hk.transform(Policy))
params_policy = policy.init(next(key), states0[0,:])

## Plotting

In [ ]:
def plot(trajs_analytical=None, trajs_naive=None, trajs_decomposition=None):

  # setup
  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # analytical
  if trajs_analytical is not None:
    for i in range(n_trajs):
      ax.plot(
        trajs_analytical[i,:1,0],
        trajs_analytical[i,:1,1],
        trajs_analytical[i,:1,2],
        'kx'
      )
      ax.plot(
        trajs_analytical[i,:,0], 
        trajs_analytical[i,:,1], 
        trajs_analytical[i,:,2], 
        'k-'
      )

  # naive
  if trajs_naive is not None:
    for i in range(n_trajs):
      ax.plot(
        trajs_naive[i,:1,0],
        trajs_naive[i,:1,1],
        trajs_naive[i,:1,2],
        'kx'
      )
      ax.plot(
        trajs_naive[i,:,0], 
        trajs_naive[i,:,1], 
        trajs_naive[i,:,2], 
        'k-'
      )

  # analytical
  if trajs_decomposition is not None:
    for i in range(n_trajs):
      ax.plot(
        trajs_decomposition[i,:1,0],
        trajs_decomposition[i,:1,1],
        trajs_decomposition[i,:1,2],
        'kx'
      )
      ax.plot(
        trajs_decomposition[i,:,0], 
        trajs_decomposition[i,:,1], 
        trajs_decomposition[i,:,2], 
        'k-'
      )

  plt.show()

## Analytical trajectories

In [ ]:
@jit
def trajectory_analytical(params, state):
  params = stop_gradient(params)
  state = stop_gradient(state)
  traj = odeint(
      lambda x, t: f_analytical(x, policy.apply(params, x)),
      state,
      times
  )
  return traj

trajectories_analytical = jit(vmap(trajectory_analytical, in_axes=(None, 0)))

In [ ]:
trajs_analytical = trajectories_analytical(params_policy, states)

In [ ]:
plot(trajs_analytical)

## Naïve trajectories

In [ ]:
@jit
def trajectory_naive(params, state):
  # params = stop_gradient(params)
  # state = stop_gradient(state)
  f = lambda x, u: f_naive.apply(params_naive, x, policy.apply(params, x))
  traj = odeint(f, state, times)
  return traj

trajectories_naive = jit(vmap(trajectory_naive, in_axes=(None, 0)))

In [ ]:
trajs_naive = trajectories_naive(params_policy, states)

In [ ]:
plot(trajs_naive)

In [ ]:
trajs_naive